In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from spectools import lpsd

import multiprocessing

In [ ]:
def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real
    
def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1/samplerate))
    f = np.zeros(samples)
    idx = np.where(np.logical_and(freqs>=min_freq, freqs<=max_freq))[0]
    f[idx] = 1
    return fftnoise(f)
    
N  = int(1e7)
fs = 2000

x1 = 200*band_limited_noise(200, 300, N, fs) + 1e-6*band_limited_noise(1, 10, N, fs) + 1e-9*band_limited_noise(0.01, 0.1, N, fs)
# x1 = np.random.rand(N)

fig, ax = plt.subplots(figsize=(16,4), dpi=150)
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')
ax.plot(x1, color='black')

f1, psd1 = signal.welch(x1, fs=fs, window=("kaiser", 30), nperseg=int(10e6), noverlap=int(10e6/2), scaling='density', return_onesided=True)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(f1, psd1, color='black')

In [ ]:
bmin = 0
order = -1
lpsd.use_c_core = False
pool = multiprocessing.Pool()

psd2 = lpsd.lpsd(x1, fs=fs, bmin=bmin, order=order, win="Kaiser", Jdes=800, psll=250, pool=pool, object_return=True, verbose=True)

In [ ]:
bmin = 0
order = -1
lpsd.use_c_core = True

psd3 = lpsd.lpsd(x1, fs=fs, bmin=bmin, order=order, win="Kaiser", Jdes=800, psll=250, object_return=True, verbose=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(f1, np.sqrt(psd1), label="Welch", color='gray')
ax.loglog(psd2.f, np.sqrt(psd2.Gxx), label="Python", color='red')
ax.loglog(psd3.f, np.sqrt(psd3.Gxx), linestyle='--', label="C w/ OpenMP", color='blue')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(f1, np.sqrt(psd1), label="Welch", color='gray')
ax.loglog(psd2.f, np.sqrt(psd2.Gxx), label="Python", color='red')
ax.loglog(psd2.f, np.sqrt(psd2.Gxy_dev), linestyle='--', label="Python (variance)", color='red')
ax.loglog(psd3.f, np.sqrt(psd3.Gxx), label="C w/ OpenMP", color='blue')
ax.loglog(psd3.f, np.sqrt(psd3.Gxy_dev), linestyle='--', label="C w/ OpenMP (variance)", color='blue')
ax.set_ylim([1e-12,0.1])
ax.set_xlim([1.5e2,4.5e2])
ax.legend(loc='upper left', bbox_to_anchor=(1, 0.5), edgecolor='black', fancybox=True, shadow=True, framealpha=1)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(f1, np.sqrt(psd1), label="Welch", color='gray')
ax.loglog(psd2.f, np.sqrt(psd2.Gxx), label="Python", color='red')
ax.loglog(psd2.f, np.sqrt(psd2.Gxy_dev), linestyle='--', label="Python (variance)", color='red')
ax.loglog(psd3.f, np.sqrt(psd3.Gxx), label="C w/ OpenMP", color='blue')
ax.loglog(psd3.f, np.sqrt(psd3.Gxy_dev), linestyle='--', label="C w/ OpenMP (variance)", color='blue')
ax.set_ylim([1e-22,1e-9])
ax.set_xlim([0.001,100])
ax.legend(loc='upper left', bbox_to_anchor=(1, 0.5), edgecolor='black', fancybox=True, shadow=True, framealpha=1)